In [1]:
%config Completer.use_jedi = False
import spacy
import re
import numpy as np
import pylab as plt
import pandas as pd
import tqdm

Idea, use the quantity to reformat the quantity into different common string representations of that float. Use those to find the bounds in the `input` string. Now let's write the functions

- There's a problem with leading and trailing whitespace

In [2]:
ingredients = pd.read_csv('../data/nyt-ingredients-snapshot-2015.csv')
ingredients.unit = ingredients.unit.str.strip()
ingredients.name = ingredients.name.str.strip()

## Quantity

In [3]:
def pluralize(noun):
    if re.search('[sxz]$', noun):
         return re.sub('$', 'es', noun)
    elif re.search('[^aeioudgkprt]h$', noun):
        return re.sub('$', 'es', noun)
    elif re.search('[aeiou]y$', noun):
        return re.sub('y$', 'ies', noun)
    else:
        return noun + 's'

In [4]:
def decimal_to_string(decimal):
    """
    """
    # Don't love this
    problem_fractions = {0.13: '1/8', 0.67: '2/3', 0.33: '1/3'}
    if decimal in problem_fractions:
        return problem_fractions.get(decimal)
    
    ratio = float(decimal).as_integer_ratio()
    return '/'.join(str(d) for d in ratio)

def string_to_decimal(string):
    """
    """
    numbers = string.split('/')
    return float(numbers[0]) / float(numbers[1])
    
def _process_and_colloquial(string):
    """
    """
    measurements = {'third': '1/3', 'half': '1/2', 'quarter': '1/4', 'fourth': '1/4'}
    splits = string.split(' and a ')
    return float(splits[0]) + measurements.get(splits[1], '')
    
def _process_and(string):
    """
    """
    splits = string.split(' and ')
    return float(splits[0]) + string_to_decimal(splits[1])

def _process(string):
    """
    """
    splits = string.split(' ')
    return float(splits[0]) + string_to_decimal(splits[1])

def string_to_float(string):
    """
    NOT GOOD NAMING
    """
    patterns = ['\d+\s(and)\s\d\/\d', '\d+\s\d\/\d']
    pattern_dict = {'\d+\s(and)\s\d\/\d': _process_and, '\d+\s\d\/\d': _process}
    
    for pattern in patterns:
        pt = re.compile(pattern)
        match = re.search(pt, string)
        if match:
            return pattern_dict[pattern](match.group())
        
    return 0

def reformat_string(string):
    """
    """
    patterns = ['\d+\s(and)\s\d\/\d', '\d+\s\d\/\d', '\d+\/\d+', '\d+']
    pattern_dict = {'\d+\s(and)\s\d\/\d': lambda x: float_to_string(_process_and(x)), 
                    '\d+\s\d\/\d': lambda x: x,
                    '\d+\/\d+': lambda x: x,
                    '\d+': lambda x: x}
    
    for pattern in patterns:
        pt = re.compile(pattern)
        match = re.search(pt, string)
        if match:
            return pattern_dict[pattern](match.group())
        
    return 0
    
            
def float_to_string(value):
    """
    """
    if value < 1:
        if (value - int(value)) < 0.001:
            return str(0)
        
        return decimal_to_string(value)
    
    if value == 1:
        return str(1)
    
    else:
        if (value - int(value)) < 0.001:
            return str(int(value))
        
        return '{} {}'.format(str(int(value)), decimal_to_string(round(value - int(value), 2)))
    
named = {1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five', 
         6: 'six', 7: 'seven', 8: 'eight', 9: 'nine'}
quarter = {0.25: 'quarter', 0.33: 'third', 0.5: 'half'}
def float_to_regex(value, error_str='-1'):
    """
    """
    if value < 1:
        if value < 0.001:
            return [str(0)]
        
        return [decimal_to_string(value)]
    
    if value == 1:
        return [str(1), 'one']
    
    else:
        if (value - int(value)) < 0.001:
            if value < 10:
                return [str(int(value)), named[int(value)]]
            
            else:
                return [str(int(value))]
        
        a = '{} {}'.format(str(int(value)), decimal_to_string(round(value - int(value), 2)))
        b = '{} and {}'.format(str(int(value)), decimal_to_string(round(value - int(value), 2)))
        c = '{} and a {}'.format(str(int(value)), quarter.get(round(value - int(value), 2), error_str))
        return [a, b, c]

In [5]:
def cleandata_debug_qty(ingredients):
    """
    """
    data = []
    for i, row in enumerate(ingredients.itertuples()):
        entry = {'entities': []}
        if pd.isna(row.input):
            continue
            
        if not pd.isna(row.qty):
            test = float_to_regex(row.qty)
            matches = [re.search(t, row.input) for t in test]
            if any(matches):
                qty = next(match for match in matches if match is not None)
                entry['entities'].append((*qty.span(), 'QUANTITY'))

    
        data.append((row.input, entry))
        
    return data

In [8]:
data = cleandata_debug_qty(ingredients)

## Unit

In [9]:
def cleandata_debug_unit(ingredients):
    """
    """
    data = []
    for i, row in enumerate(ingredients.itertuples()):
        entry = {'entities': []}
        if pd.isna(row.input):
            continue
            
        try:
            if not pd.isna(row.unit):
                unit = re.search(row.unit, row.input, re.IGNORECASE)
                entry['entities'].append((*unit.span(), 'UNIT'))
                
                data.append((row.input, entry))
            
        except Exception as e:
            pass
        
    
    return data

In [10]:
data = cleandata_debug_unit(ingredients)

## Product

In [11]:
def cleandata_debug_name(ingredients):
    """
    """
    data = []
    rows = []
    for i, row in enumerate(ingredients.itertuples()):
        entry = {'entities': []}
        if pd.isna(row.input):
            continue
        
        try:
            if not pd.isna(row.name):
                product = re.search(row.name, row.input, re.IGNORECASE)
                if product:
                    entry['entities'].append((*product.span(), 'PRODUCT'))
        except Exception as e:
            rows.append((i, e, row))
                
        data.append((row.input, entry))
    
    return rows

In [12]:
data = cleandata_debug_name(ingredients)

# Bring it all together

In [15]:
# This function is horrendous
def cleandata(ingredients):
    """
    """
    data = []
    for i, row in enumerate(ingredients.itertuples()):
        entry = {'entities': []}
        if pd.isna(row.input):
            continue
            
        # Quantity
        if not pd.isna(row.qty):
            test = float_to_regex(row.qty)
            matches = [re.search(t, row.input) for t in test]
            if any(matches):
                qty = next(match for match in matches if match is not None)
                entry['entities'].append((*qty.span(), 'QUANTITY'))
         
        # Product
        try:
            if not pd.isna(row.name):
                product = re.search(row.name, row.input, re.IGNORECASE)
                if product:
                    entry['entities'].append((*product.span(), 'PRODUCT'))
        except Exception as e:
            pass
        
        
        # Unit
        try:
            if not pd.isna(row.unit):
                unit = re.search(row.unit, row.input, re.IGNORECASE)
                entry['entities'].append((*unit.span(), 'UNIT'))
            
        except Exception as e:
            pass

    
        data.append((row.input, entry))
        
    return data

In [16]:
datacleaned = cleandata(ingredients)

## Overlapping String Problems

In [17]:
# Please rewrite this this is so embarassing
def overlap(data):
    store = []
    for i, row in enumerate(data):
        pairs = row[1]['entities']
        for p1 in pairs:
            for p2 in pairs:
                if p1 == p2:
                    continue

                condition_a = (p1[0] > p2[0] and p1[1] < p2[1])
                condition_b = (p1[0] <= p2[0] and p1[1] >= p2[0])
                condition_c = (p1[0] == p2[0] or p1[1] == p2[1])
                if condition_a or condition_b or condition_c:
                    store.append(i)
                    
    return list(set(store))

In [23]:
dupes = overlap(datacleaned)
DATA = [datacleaned[i] for i in range(len(datacleaned)) if i not in dupes]

## Train Model

In [75]:
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training.example import Example
import warnings
warnings.filterwarnings("ignore")

In [76]:
nlp = spacy.load('en_core_web_sm')
ner = nlp.get_pipe("ner")

In [77]:
for _, annotations in DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

In [78]:
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [82]:
TRAINING_DATA = DATA[:int(len(DATA) * 0.5)]
BATCH_SIZE = 10

stores = []
losses = {}
for batch in tqdm.tqdm(minibatch(TRAINING_DATA, size=BATCH_SIZE), total=len(TRAINING_DATA) // BATCH_SIZE):
    for text, annotations in batch:
        # create Example
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        # Update the model
        nlp.update([example], losses=losses, drop=0.3)
        
    stores.append(losses)

8950it [46:03,  3.24it/s]                           


In [83]:
doc = nlp(DATA[19000][0])
print (DATA[19000][0])
print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

1/4 teaspoon dried thyme
Entities [('1/4', 'QUANTITY'), ('teaspoon', 'UNIT'), ('thyme', 'PRODUCT')]


In [84]:
string = ingredients.loc[ingredients.input.str.len() == ingredients.input.str.len().max()].input.iloc[0]
doc = nlp(string)
print (string)
print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

2 pounds kabocha or butternut squash, peeled and cut in small dice (about 3 cups peeled and diced, weighing 1 1/2 to 1 3/4 pounds), or 1 large acorn squash, cut in half \n\n(If using acorn squash, place in a 425 degree oven for 20 minutes before cutting in 
Entities [('2', 'QUANTITY'), ('kabocha', 'PRODUCT'), ('squash', 'PRODUCT'), ('squash', 'PRODUCT')]


In [85]:
output_dir = '/Users/tyler/Projects/recipe-parser/recipe-parser/ingredient_parser/models/'

if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Saved model to /Users/tyler/Projects/recipe-parser/recipe-parser/ingredient_parser/models
